<a href="https://colab.research.google.com/github/CHIN-HUA/AI/blob/main/Parallelization_of_Graph_Convolutional_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-11.8  games	       include	lib64	   man	 share
colab  cuda-11	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [ ]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Mon May 15 14:41:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Write a cu file contain the host and kernel code
%%writefile hello.cu

#include <iostream>
#include <cuda.h>

__global__ void hello(void)
{
  printf("GPU: Hello!\n");
}
int main(int argc,char **argv)
{
  printf("CPU: Hello!\n");
  hello<<<1,10>>>();
  cudaDeviceReset();
  return 0;
}

Overwriting hello.cu


In [ ]:
!nvcc -Wno-deprecated-gpu-targets hello.cu -o hello

!./hello

CPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!


In [ ]:
'''
MIT License
Copyright (c) 2020 Ronald Seoh
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
'''

# torch.device / CUDA Setup
import torch

use_cuda = True
use_colab_tpu = False
colab_tpu_available = False

if use_colab_tpu:
    try:
        assert os.environ['COLAB_TPU_ADDR']
        colab_tpu_available = True
    except:
        colab_tpu_available = True

if use_cuda and torch.cuda.is_available():
    torch_device = torch.device('cuda')

    # Set this to True to make your output immediately reproducible
    # Note: https://pytorch.org/docs/stable/notes/randomness.html
    torch.backends.cudnn.deterministic = False
    
    # Disable 'benchmark' mode: Set this False if you want to measure running times more fairly
    # Note: https://discuss.pytorch.org/t/what-does-torch-backends-cudnn-benchmark-do/5936
    torch.backends.cudnn.benchmark = True
    
    # Faster Host to GPU copies with page-locked memory
    use_pin_memory = True 

    # CUDA libraries version information
    print("CUDA Version: " + str(torch.version.cuda))
    print("cuDNN Version: " + str(torch.backends.cudnn.version()))
    print("CUDA Device Name: " + str(torch.cuda.get_device_name()))
    print("CUDA Capabilities: "+ str(torch.cuda.get_device_capability()))

elif use_colab_tpu and colab_tpu_available:
    # This needs to be installed separately
    # https://github.com/pytorch/xla/blob/master/contrib/colab/getting-started.ipynb
    import torch_xla 
    import torch_xla.core.xla_model as xm

    torch_device = xm.xla_device()

else:
    torch_device = torch.device('cpu')
    use_pin_memory = False

CUDA Version: 11.8
cuDNN Version: 8700
CUDA Device Name: Tesla T4
CUDA Capabilities: (7, 5)


In [ ]:
!git clone https://github.com/hengdashi/cuda_gcn.git

Cloning into 'cuda_gcn'...
remote: Enumerating objects: 425, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 425 (delta 4), reused 13 (delta 4), pack-reused 410
Receiving objects: 100% (425/425), 21.89 MiB | 28.93 MiB/s, done.
Resolving deltas: 100% (230/230), done.


In [ ]:
!ls ./cuda_gcn/

data.tgz  README.md	     reddit_preprocess.py  src
Makefile  reddit_edges.json  report


In [ ]:
!ls
!make cuda

cuda_gcn  Makefile   reddit_edges.json	   report   src
data.tgz  README.md  reddit_preprocess.py  seq_gcn
nvcc -O3 -std=c++11 -Isrc -Isrc/common -Isrc/seq -Isrc/cuda src/cudamain.o src/common/parser.o src/common/timer.o src/seq/gcn.o src/seq/module.o src/seq/optim.o src/seq/rand.o src/seq/sparse.o src/seq/variable.o src/cuda/cuda_gcn.o src/cuda/cuda_kernel.o src/cuda/cuda_module.o src/cuda/cuda_variable.o -o cuda_gcn


In [53]:
!ls
!tar -xvzf data.tgz

!./cuda_gcn cora

cuda_gcn  data.tgz  README.md	       reddit_preprocess.py  seq_gcn
data	  Makefile  reddit_edges.json  report		     src
data/citeseer.graph
data/citeseer.split
data/citeseer.svmlight
data/cora.graph
data/cora.split
data/cora.svmlight
data/pubmed.graph
data/pubmed.split
data/pubmed.svmlight
Parse Graph Succeeded.
Parse Node Succeeded.
Parse Split Succeeded.
RUNNING ON GPU
epoch=1 train_loss=1.95439 train_acc=0.11429 val_loss=1.95068 val_acc=0.17200 time=0.00282
epoch=2 train_loss=1.94962 train_acc=0.22857 val_loss=1.94731 val_acc=0.33400 time=0.00259
epoch=3 train_loss=1.94452 train_acc=0.42857 val_loss=1.94432 val_acc=0.48400 time=0.00254
epoch=4 train_loss=1.94004 train_acc=0.61429 val_loss=1.94142 val_acc=0.55600 time=0.00258
epoch=5 train_loss=1.93708 train_acc=0.59286 val_loss=1.93847 val_acc=0.57400 time=0.00257
epoch=6 train_loss=1.92908 train_acc=0.68571 val_loss=1.93540 val_acc=0.59200 time=0.00255
epoch=7 train_loss=1.92625 train_acc=0.67857 val_loss=1.93241 val_acc=0.57200 ti